# Sensor Bulk Load Review

In [1]:
# Import likely important packages, etc.
import sys, os, csv, fnmatch, re
import datetime
import time
import xml.etree.ElementTree as et
from zipfile import ZipFile
import numpy as np
import pandas as pd
import xarray as xr

In [4]:
def whoi_asset_tracking(spreadsheet,sheet_name,instrument_class='All',whoi=True,series=None):
    """
    Loads all the individual sensors of a specific instrument class and
    series type. Currently applied only for WHOI deployed instruments.
    
    Args:
        spreadsheet - directory path and name of the excel spreadsheet with
            the WHOI asset tracking information.
        sheet_name - name of the sheet in the spreadsheet to load
        instrument_class - the type (i.e. CTDBP, CTDMO, PCO2W, etc). Defaults
            to 'All', which will load all of the instruments
        whoi - return only whoi instruments? Defaults to True.
        series - a specified class of the instrument to load. Defaults to None,
            which will load all of the series for a specified instrument class
    """
    
    all_sensors = pd.read_excel(spreadsheet,sheet_name=sheet_name,header=1)
    # Select a specific class of instruments
    if instrument_class == 'All':
        inst_class = all_sensors
    else:
        inst_class  = all_sensors[all_sensors['Instrument\nClass']==instrument_class]
    # Return only the whoi instruments?
    if whoi == True:
        whoi_insts = inst_class[inst_class['Deployment History'] != 'EA']
    else:
        whoi_insts = inst_class
    # Slect a specific series of the instrument?
    if series != None:
        instrument = whoi_insts[whoi_insts['Series'] == series]
    else:
        instrument = whoi_insts
 
    return instrument

In [5]:
excel_spreadsheet = 'C:/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
#excel_spreadsheet = '/media/andrew/OS/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
sheet_name = 'Sensors'

In [6]:
instruments = whoi_asset_tracking(excel_spreadsheet,sheet_name)
instruments

,Instrument Class,Series,Supplier Serial Number,WHOI #,OOI #,UID,Model,CGSN PN,Firmware Version,Supplier,...,QCT Testing,PreDeployment,Post Deployment,Refurbishment/ Repair,DO Number,Date Received,Deployment History,Current Deployment,Instrument Location on Current Deployment,Notes
0,ADCPS,L,18260,115099,A00075,CGINS-ADCPSL-18260,WHLS75-1500-I,1336-00010-00012,50.4,Teledyne,...,3305-00107-00015\n3305-00107-00114\n3305-00107...,3305-00307-00047\n3305-00307-00191,NaN,3305-00900-00047,WH-SC12-10-ADCP-1001,09/25/2012,GP03FLMB-00001\nGP03FLMA-00004,NaN,NaN,NaN
1,ADCPS,L,18261,115100\n118309,A00076,CGINS-ADCPSL-18261,WHLS75-1500-I,1336-00010-00012,NaN,Teledyne,...,3305-00107-00014\n3305-00107-00116,3305-00307-00077,NaN,NaN,WH-SC12-10-ADCP-1001,09/25/2012,GI03FLMA-00001\nGP03FLMB-00005,NaN,NaN,NaN
2,ADCPS,L,18351,115091,A00068,CGINS-ADCPSL-18351,WHLS75-1500-I,1336-00010-00012,50.4,Teledyne,...,3305-00107-00009\n3305-00107-00016\n3305-00107...,3305-00307-00055\n3305-00307-00160,NaN,NaN,WH-SC12-10-ADCP-1001,2012-10-02 00:00:00,GP03FLMA-00001\nGP03FLMB-00004,GI03FLMB-00005,500m,"Delivered to WHOI, Shipped to SIO"
3,ADCPS,L,18352,115090,A00067,CGINS-ADCPSL-18352,WHLS75-1500-I,1336-00010-00012,50.4,Teledyne,...,3305-00107-00011\n3305-00107-00017\n3305-00107...,3305-00307-00078,NaN,3305-00900-00046\n3305-00900-00347,WH-SC12-10-ADCP-1001,2012-10-02 00:00:00,GA03FLMA-00003,NaN,NaN,Sent from SIO to WHOI\nFlooded on Argentine de...
4,ADCPS,L,18353,115101,A00077,CGINS-ADCPSL-18353,WHLS75-1500-I,1336-00010-00012,50.4,Teledyne,...,3305-00107-00018\n3305-00107-00117\n3305-00107...,3305-00307-00075\n3305-00307-00146,NaN,3305-00900-00427,WH-SC12-10-ADCP-1001,09/25/2012,GI03FLMB-00001\nGI03FLMA-00003\nCP02PMUO-00011,NaN,NaN,NaN
5,ADCPS,L,18445,115092,A00069,CGINS-ADCPSL-18445,WHLS75-1500-I,1336-00010-00012,50.4,Teledyne,...,3305-00107-00010\n3305-00107-00096,3305-00307-00012\n3305-00307-00043\n3305-00307...,NaN,NaN,WH-SC12-10-ADCP-1001,2012-10-02 00:00:00,CP02PMUO-00001\nCP02PMUO-00002\nCP02PMUO-00004...,NaN,NaN,Batteries Replaced 9/28/15
6,ADCPS,L,18446,115093,A00070,CGINS-ADCPSL-18446,WHLS75-1500-I,1336-00010-00012,50.4,Teledyne,...,3305-00107-00008\n3305-00107-00049,3305-00307-00014\n3305-00307-00022\n3305-00307...,NaN,NaN,WH-SC12-10-ADCP-1001,2012-10-02 00:00:00,CP02PMUO-00005\nCP02PMUO-00007\nGS03FLMA-00003,NaN,NaN,Might not be deployed on UO7 if the endcap doe...
7,ADCPS,J,18976,115293,A00182,CGINS-ADCPSJ-18976,WHLS75-1500,1336-00010-00010,50.4,Teledyne,...,3305-00107-00026\n3305-00107-00079\n3305-00107...,3305-00307-00095\n3305-00307-00174,NaN,3305-00900-00002,WH-SC12-10-ADCP-1004,2013-01-25 00:00:00,CP4 Spare\nCP04OSSM-00006\nCP04OSSM-00009,CP04OSSM-00009,NaN,Bent Pins. End-cap sent back to vendor for par...
8,ADCPS,L,20346,115761,A00439,CGINS-ADCPSL-20346,WHLS75-1500-I,1336-00010-00012,50.4,Teledyne,...,3305-00107-00040\n3305-00107-00100\n3305-00107...,3305-00307-00005\n3305-00307-00080\n3305-00307...,NaN,3305-00900-00025\n3305-00900-00181,WH-SC12-10-ADCP-1005,2013-09-16 00:00:00,CP02PMUO-00003\nCP4 Spare\nCP02PMUO-00007\nCP0...,NaN,NaN,Took batteries from unit to put on other ADCPs...
9,ADCPS,L,20502,115925,A00568,CGINS-ADCPSL-20502,WHLS75-1500-I,1336-00010-00012,NaN,Teledyne,...,3305-00107-00039\n3305-00107-00093,3305-00307-00040\n3305-00307-00159,NaN,NaN,WH-SC12-10-ADCP-1005,11/15/2013,GP03FLMA-00002\nGS03FLMA-00002,GI03FLMA-00005,NaN,NaN


In [47]:
# Now I need to load the sensor bulk load info
bulkload_spreadsheet = 'C:/Users/areed/Documents/OOI-CGSN/GitHub/OOI-Integration/asset-management/bulk/sensor_bulk_load-AssetRecord.csv'

In [48]:
bulkload = pd.read_csv(bulkload_spreadsheet)
bulkload

,ASSET_UID,LEGACY_ASSET_UID,TYPE,Mobile,DESCRIPTION OF EQUIPMENT,MIO_Inventory_Description,Manufacturer,Model,Manufacturer's Serial No./Other Identifier,Firmware Version,ACQUISITION DATE,ORIGINAL COST,comments,MIO
0,#ATAPL-69140-001-120-1,ATAPL-69140-001-120-1,Sensor,0,Shallow Profiler Winch Controller,Winch Controller,APL,Winch Controller,120,NaN,NaN,NaN,RS01SBPS-SC01A-00-WINCH,University of Washington
1,#ATAPL-69140-001-120-2,ATAPL-69140-001-120-2,Sensor,0,Shallow Profiler Winch Controller Node Controller,Node Controller,APL,Node Controller,120,NaN,NaN,NaN,RS01SBPS-SC01A-00-ENG,University of Washington
2,A00835,A00835,Sensor,0,SENSOR CTD,SENSOR CTD,Sea-Bird Electronics,SBE 52-MP,138,NaN,20140128.0,9966.0,NaN,NaN
3,A00838,A00838,Sensor,0,FLUOROMETER,FLUOROMETER,WET Labs,FLCDRTD (CDOM),3400,NaN,20140128.0,6665.0,UW Claims they have asset,NaN
4,A01061,A01061,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 1,NaN,20140423.0,173968.0,NaN,NaN
5,A01062,A01062,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 2,NaN,20140423.0,173968.0,NaN,NaN
6,A01073,A01073,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 3,NaN,20140423.0,173968.0,NaN,NaN
7,A01085,A01085,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141004,NaN,20140304.0,4565.0,NaN,NaN
8,A01086,A01086,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141005,NaN,20150604.0,4565.0,NaN,NaN
9,A01088,A01088,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141009,NaN,20140304.0,4108.5,NaN,NaN


In [49]:
# Now remove the RCA instruments from the sensor bulk load
bulkload['RCA'] = bulkload['ASSET_UID'].apply(lambda x: True if 'APL' in x else False)
bulkload

,ASSET_UID,LEGACY_ASSET_UID,TYPE,Mobile,DESCRIPTION OF EQUIPMENT,MIO_Inventory_Description,Manufacturer,Model,Manufacturer's Serial No./Other Identifier,Firmware Version,ACQUISITION DATE,ORIGINAL COST,comments,MIO,RCA
0,#ATAPL-69140-001-120-1,ATAPL-69140-001-120-1,Sensor,0,Shallow Profiler Winch Controller,Winch Controller,APL,Winch Controller,120,NaN,NaN,NaN,RS01SBPS-SC01A-00-WINCH,University of Washington,True
1,#ATAPL-69140-001-120-2,ATAPL-69140-001-120-2,Sensor,0,Shallow Profiler Winch Controller Node Controller,Node Controller,APL,Node Controller,120,NaN,NaN,NaN,RS01SBPS-SC01A-00-ENG,University of Washington,True
2,A00835,A00835,Sensor,0,SENSOR CTD,SENSOR CTD,Sea-Bird Electronics,SBE 52-MP,138,NaN,20140128.0,9966.0,NaN,NaN,False
3,A00838,A00838,Sensor,0,FLUOROMETER,FLUOROMETER,WET Labs,FLCDRTD (CDOM),3400,NaN,20140128.0,6665.0,UW Claims they have asset,NaN,False
4,A01061,A01061,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 1,NaN,20140423.0,173968.0,NaN,NaN,False
5,A01062,A01062,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 2,NaN,20140423.0,173968.0,NaN,NaN,False
6,A01073,A01073,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 3,NaN,20140423.0,173968.0,NaN,NaN,False
7,A01085,A01085,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141004,NaN,20140304.0,4565.0,NaN,NaN,False
8,A01086,A01086,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141005,NaN,20150604.0,4565.0,NaN,NaN,False
9,A01088,A01088,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141009,NaN,20140304.0,4108.5,NaN,NaN,False


In [50]:
# Now pull out the CG/EA bulkload
CGEA_bulkload = bulkload[bulkload['RCA'] == False]
CGEA_bulkload

,ASSET_UID,LEGACY_ASSET_UID,TYPE,Mobile,DESCRIPTION OF EQUIPMENT,MIO_Inventory_Description,Manufacturer,Model,Manufacturer's Serial No./Other Identifier,Firmware Version,ACQUISITION DATE,ORIGINAL COST,comments,MIO,RCA
2,A00835,A00835,Sensor,0,SENSOR CTD,SENSOR CTD,Sea-Bird Electronics,SBE 52-MP,138,NaN,20140128.0,9966.0,NaN,NaN,False
3,A00838,A00838,Sensor,0,FLUOROMETER,FLUOROMETER,WET Labs,FLCDRTD (CDOM),3400,NaN,20140128.0,6665.0,UW Claims they have asset,NaN,False
4,A01061,A01061,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 1,NaN,20140423.0,173968.0,NaN,NaN,False
5,A01062,A01062,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 2,NaN,20140423.0,173968.0,NaN,NaN,False
6,A01073,A01073,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 3,NaN,20140423.0,173968.0,NaN,NaN,False
7,A01085,A01085,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141004,NaN,20140304.0,4565.0,NaN,NaN,False
8,A01086,A01086,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141005,NaN,20150604.0,4565.0,NaN,NaN,False
9,A01088,A01088,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141009,NaN,20140304.0,4108.5,NaN,NaN,False
10,A01330,A01330,Sensor,0,VELPT SERIES A (AQUADOPP 300M),VELPT SERIES A (AQUADOPP 300M),Nortek,Aquadopp,P21893-1,NaN,20150402.0,8900.0,NaN,NaN,False
11,A01331,A01331,Sensor,0,VELPT SERIES B (AQUADOPP 300M),VELPT SERIES B (AQUADOPP 300M),Nortek,Aquadopp,P28193-3,NaN,20150402.0,9925.0,NaN,NaN,False


In [52]:
# Now cycle through the bulkload and check if the instrument is listed in whoi asset tracking. Mark if it not.
uids = list(instruments['UID'])
uids

['CGINS-ADCPSL-18260',
 'CGINS-ADCPSL-18261',
 'CGINS-ADCPSL-18351',
 'CGINS-ADCPSL-18352',
 'CGINS-ADCPSL-18353',
 'CGINS-ADCPSL-18445',
 'CGINS-ADCPSL-18446',
 'CGINS-ADCPSJ-18976',
 'CGINS-ADCPSL-20346',
 'CGINS-ADCPSL-20502',
 'CGINS-ADCPSL-20503',
 'CGINS-ADCPSL-20504',
 'CGINS-ADCPSN-21002',
 'CGINS-ADCPSL-21393',
 'CGINS-ADCPSL-21446',
 'CGINS-ADCPSL-21450',
 'CGINS-ADCPSJ-21496',
 'CGINS-ADCPSL-21497',
 'CGINS-ADCPSN-21638',
 'CGINS-ADCPSN-21639',
 'CGINS-ADCPSN-21654',
 'CGINS-ADCPSN-21717',
 'CGINS-ADCPSJ-22642',
 'CGINS-ADCPSL-22650',
 'CGINS-ADCPSL-22846',
 'CGINS-ADCPSN-22865',
 'CGINS-ADCPSL-23380',
 'CGINS-ADCPSL-23381',
 'CGINS-ADCPSL-23497',
 'CGINS-ADCPSL-23498',
 'CGINS-ADCPSN-23579',
 'CGINS-ADCPSN-23580',
 'CGINS-ADCPSL-24661',
 'CGINS-ADCPSL-24664',
 'CGINS-ADCPTF-18593',
 'CGINS-ADCPTF-18594',
 'CGINS-ADCPTG-18596',
 'CGINS-ADCPTG-18660',
 'CGINS-ADCPTG-19151',
 'CGINS-ADCPTF-19221',
 'CGINS-ADCPTG-19336',
 'CGINS-ADCPTG-20495',
 'CGINS-ADCPTG-20496',
 'CGINS-ADC

In [53]:
CGEA_bulkload['In Asset Tracking'] = CGEA_bulkload['ASSET_UID'].apply(lambda x: True if x in uids else False)
CGEA_bulkload

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ASSET_UID,LEGACY_ASSET_UID,TYPE,Mobile,DESCRIPTION OF EQUIPMENT,MIO_Inventory_Description,Manufacturer,Model,Manufacturer's Serial No./Other Identifier,Firmware Version,ACQUISITION DATE,ORIGINAL COST,comments,MIO,RCA,In Asset Tracking
2,A00835,A00835,Sensor,0,SENSOR CTD,SENSOR CTD,Sea-Bird Electronics,SBE 52-MP,138,NaN,20140128.0,9966.0,NaN,NaN,False,False
3,A00838,A00838,Sensor,0,FLUOROMETER,FLUOROMETER,WET Labs,FLCDRTD (CDOM),3400,NaN,20140128.0,6665.0,UW Claims they have asset,NaN,False,False
4,A01061,A01061,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 1,NaN,20140423.0,173968.0,NaN,NaN,False,False
5,A01062,A01062,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 2,NaN,20140423.0,173968.0,NaN,NaN,False,False
6,A01073,A01073,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 3,NaN,20140423.0,173968.0,NaN,NaN,False,False
7,A01085,A01085,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141004,NaN,20140304.0,4565.0,NaN,NaN,False,False
8,A01086,A01086,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141005,NaN,20150604.0,4565.0,NaN,NaN,False,False
9,A01088,A01088,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141009,NaN,20140304.0,4108.5,NaN,NaN,False,False
10,A01330,A01330,Sensor,0,VELPT SERIES A (AQUADOPP 300M),VELPT SERIES A (AQUADOPP 300M),Nortek,Aquadopp,P21893-1,NaN,20150402.0,8900.0,NaN,NaN,False,False
11,A01331,A01331,Sensor,0,VELPT SERIES B (AQUADOPP 300M),VELPT SERIES B (AQUADOPP 300M),Nortek,Aquadopp,P28193-3,NaN,20150402.0,9925.0,NaN,NaN,False,False


In [54]:
# Print out the Asset UID
not_in_asset_tracking = CGEA_bulkload[CGEA_bulkload['In Asset Tracking'] == False]
not_in_asset_tracking

,ASSET_UID,LEGACY_ASSET_UID,TYPE,Mobile,DESCRIPTION OF EQUIPMENT,MIO_Inventory_Description,Manufacturer,Model,Manufacturer's Serial No./Other Identifier,Firmware Version,ACQUISITION DATE,ORIGINAL COST,comments,MIO,RCA,In Asset Tracking
2,A00835,A00835,Sensor,0,SENSOR CTD,SENSOR CTD,Sea-Bird Electronics,SBE 52-MP,138,NaN,20140128.0,9966.0,NaN,NaN,False,False
3,A00838,A00838,Sensor,0,FLUOROMETER,FLUOROMETER,WET Labs,FLCDRTD (CDOM),3400,NaN,20140128.0,6665.0,UW Claims they have asset,NaN,False,False
4,A01061,A01061,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 1,NaN,20140423.0,173968.0,NaN,NaN,False,False
5,A01062,A01062,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 2,NaN,20140423.0,173968.0,NaN,NaN,False,False
6,A01073,A01073,Sensor,0,SONAR BIO ACOUSTIC,SONAR BIO ACOUSTIC,Kongsberg Underwater Technology Inc,CLIN 001,S/N 3,NaN,20140423.0,173968.0,NaN,NaN,False,False
7,A01085,A01085,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141004,NaN,20140304.0,4565.0,NaN,NaN,False,False
8,A01086,A01086,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141005,NaN,20150604.0,4565.0,NaN,NaN,False,False
9,A01088,A01088,Sensor,0,ANEMOMETER WINDMASTER PRO,ANEMOMETER WINDMASTER PRO,PP SYSTEMS,WINDMASTER PRO,TAS141009,NaN,20140304.0,4108.5,NaN,NaN,False,False
10,A01330,A01330,Sensor,0,VELPT SERIES A (AQUADOPP 300M),VELPT SERIES A (AQUADOPP 300M),Nortek,Aquadopp,P21893-1,NaN,20150402.0,8900.0,NaN,NaN,False,False
11,A01331,A01331,Sensor,0,VELPT SERIES B (AQUADOPP 300M),VELPT SERIES B (AQUADOPP 300M),Nortek,Aquadopp,P28193-3,NaN,20150402.0,9925.0,NaN,NaN,False,False


In [55]:
not_in_asset_tracking.to_csv('sensor_bulk_load_not_in_asset_tracking.csv')